# 번호판 인식 모델

In [20]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import re
import math
import tensorflow as tf


# ------------ 번호판 ------------
# img = './yolov5/runs/hub/exp/crops/license_plate/v01.jpg' # X
# img = './yolov5/runs/hub/exp/crops/license_plate/v02.jpg' # X
# img = './yolov5/runs/hub/exp/crops/license_plate/v03.jpg' # 0 blur_2
img = './yolov5/runs/hub/exp/crops/license_plate/v04.jpg' # 0 blur_2


def OCR_pn(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # ------------------- 기울기 조정 start -------------------
    canny = cv2.Canny(blur, 700, 350, apertureSize = 5, L2gradient = True)
    lines = cv2.HoughLinesP(canny, 1, np.pi / 180, 50, minLineLength = 3, maxLineGap = 150)

    angle = 0
    maxdim = 0
    if not (lines is None):
        for i in lines:
            xdim = i[0][2] - i[0][0]
            ydim = i[0][3] - i[0][1]
            iangle = math.atan2(ydim, xdim)*180/np.pi
            dim = math.sqrt((xdim * xdim) + (ydim * ydim))
            if abs(angle) < 40 and maxdim < dim:
                maxdim = dim
                angle =iangle

    roih, roiw, roic = img.shape
    matrix = cv2.getRotationMatrix2D((roiw/2, roih/2), angle, 1)
    roi = cv2.warpAffine(img, matrix, (roiw, roih))
    # ------------------- 기울기 조정 fin -------------------

    gray_2 = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    gray_2 = cv2.resize(gray_2, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur_2 = cv2.GaussianBlur(gray_2, (5,5), 0)

    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(blur_2, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    blur_3 = cv2.GaussianBlur(dilation, (5,5), 0)

    plate_num = ""

    try:
        text = pytesseract.image_to_string(blur_2, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')

        plate_num = re.sub('[\W_]+', '', text)            
    except:
        text = None
        
    if plate_num != None:
        return plate_num[-4:]
    else:
        return False

OCR_pn()

'6259'

# 옆번호 인식 모델

In [22]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import re
import math
import tensorflow as tf


# ------------ 옆번호 ------------
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v06.jpg' # X
img = './yolov5/runs/hub/exp/crops/bus_num_fr/v07.jpg' # 0 blur_2 (임계값 처리)
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v08.jpg' # X
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v09.jpg' # X
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v10.jpg' # X
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v11.jpg' # X
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v12.jpg' # X
# img = './yolov5/runs/hub/exp/crops/bus_num_fr/v13.jpg' # X

def OCR_bn(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    blur_2 = cv2.GaussianBlur(dilation, (5,5), 0)

    plate_num = ""
    
    try:
        text = pytesseract.image_to_string(blur_2, config='-c tessedit_char_whitelist=0123456789 --psm 7 --oem 1')

        bus_num = re.sub('[\W_]+', '', text)            
    except:
        text = None
        
    if bus_num != None:
        return bus_num[-4:]
    else:
        return False
    
OCR_bn(img)

<class 'str'>
